In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load your data
df = pd.read_csv("jobs_in_data.csv")

# Define features and target variable
X = df["job_title"]  # Assuming "job_title" contains the text data
y = df["salary"]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize input text data
max_length = 64  # Maximum sequence length for BERT
xtrain_tokenized = tokenizer(X_train.tolist(), padding=True, truncation=True, max_length=max_length, return_tensors="tf")
xtest_tokenized = tokenizer(X_test.tolist(), padding=True, truncation=True, max_length=max_length, return_tensors="tf")

# Load pre-trained BERT model
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

# Freeze BERT layers (optional)
bert_model.trainable = False

# Extract BERT embeddings
xtrain_bert_output = bert_model(xtrain_tokenized)
xtest_bert_output = bert_model(xtest_tokenized)

# Aggregate BERT embeddings
xtrain_aggregated = GlobalAveragePooling1D()(xtrain_bert_output.last_hidden_state)
xtest_aggregated = GlobalAveragePooling1D()(xtest_bert_output.last_hidden_state)

# Define and compile the model
input_layer = Input(shape=(bert_model.config.hidden_size,))
output_layer = Dense(1, activation='linear')(input_layer)
model = Model(inputs=input_layer, outputs=output_layer)
optimizer = Adam(learning_rate=1e-5)  # Adjust learning rate as needed
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model
model.fit(xtrain_aggregated, y_train, epochs=5, batch_size=32, validation_split=0.1)

# Evaluate the model
y_pred = model.predict(xtest_aggregated)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", np.sqrt(mse))


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/5
211/211 [==============================] - 2s 5ms/step - loss: 26334900224.0000 - val_loss: 26363328512.0000
Epoch 2/5
211/211 [==============================] - 1s 4ms/step - loss: 26334806016.0000 - val_loss: 26363228160.0000
Epoch 3/5
211/211 [==============================] - 1s 4ms/step - loss: 26334703616.0000 - val_loss: 26363127808.0000
Epoch 4/5
211/211 [==============================] - 1s 4ms/step - loss: 26334601216.0000 - val_loss: 26363031552.0000
Epoch 5/5
59/59 [==============================] - 1s 3ms/step
Mean Squared Error: 27267752512.135155
Root Mean Squared Error: 165129.50224637374
